In [ ]:
import sys
sys.path.append('../common')
from env_variables import *

In [ ]:
# Manifest filenames
templateManifestFileName = "../common/deployment.lva_common.template.json"
deploymentManifestFileName = "deployment.lva_openvino.template.json"

In [ ]:
from shutil import copyfile

# Create a copy from the template
copyfile(templateManifestFileName, deploymentManifestFileName)

In [ ]:
LAST STOPPED HERE

Update Manifest File with AI Module Settings

In [ ]:
import json

with open(deploymentManifestFileName) as f:
    deploymentManifest = json.load(f)

aiModule = deploymentManifest["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"]["<AIX_MODULE>"]
del deploymentManifest["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"]["<AIX_MODULE>"]

aiModuleS = json.dumps(aiModule)
if useACRFlag:
    aiModuleS = aiModuleS.replace('AIX_CONTAINER_REGISTRY_FULL_NAME', containerRegServiceName+".azurecr.io")
else:
    aiModuleS = aiModuleS.replace('AIX_CONTAINER_REGISTRY_FULL_NAME', localContainerRegServiceName)
aiModuleS = aiModuleS.replace('AIX_CONTAINER_MODULE_NAME', containerImageName)

aiModule = json.loads(aiModuleS)

deploymentManifest["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"][containerImageName] = aiModule

with open(deploymentManifestFileName, "w") as f:
    deploymentManifest = json.dump(deploymentManifest, f, indent=4, sort_keys=True)

Update Manifest File with LVA Module Settings

In [ ]:
import json

with open(deploymentManifestFileName) as f:
    deploymentManifest = json.load(f)

lvaModule = deploymentManifest["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"]["<LVA_MODULE>"]
del deploymentManifest["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"]["<LVA_MODULE>"]

lvaModuleS = json.dumps(lvaModule)

lvaModule = json.loads(lvaModuleS)

for s in lvaSettings:
    deploymentManifest["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"]["lva" + s[0][-3:]] = lvaModule

with open(deploymentManifestFileName, "w") as f:
    deploymentManifest = json.dump(deploymentManifest, f, indent=4, sort_keys=True)

Set Module Properties in the Deployment Manifest File

In [ ]:
import json

with open(deploymentManifestFileName) as f:
    deploymentManifest = json.load(f)

aiModuleProp = deploymentManifest["modulesContent"]["<AIX_MODULE_PROPERTIES>"]
del deploymentManifest["modulesContent"]["<AIX_MODULE_PROPERTIES>"]
deploymentManifest["modulesContent"][containerImageName] = aiModuleProp

with open(deploymentManifestFileName, "w") as f:
    deploymentManifest = json.dump(deploymentManifest, f, indent=4, sort_keys=True)

In [ ]:
with open(deploymentManifestFileName) as f:
    deploymentManifest = json.load(f)

lvaModule = deploymentManifest["modulesContent"]["<LVA_MODULE_PROPERTIES>"]
del deploymentManifest["modulesContent"]["<LVA_MODULE_PROPERTIES>"]


for s in lvaSettings:
    lvaModuleS = json.dumps(lvaModule)
    lvaModuleS = lvaModuleS.replace('AIX_IP_CAM_ADDRESS', s[0])
    lvaModuleS = lvaModuleS.replace('AIX_IP_CAM_USER_NAME', s[1])
    lvaModuleS = lvaModuleS.replace('AIX_IP_CAM_PASSWORD', s[2])
    lvaModuleS = lvaModuleS.replace('AIX_MOTION_SENSITIVITY', s[3])
    lvaModuleS = lvaModuleS.replace('AIX_IP_CAM_FPS_FILTER', s[4])
    temp = json.loads(lvaModuleS)

    deploymentManifest["modulesContent"]["lva" + s[0][-3:]] = temp


with open(deploymentManifestFileName, "w") as f:
    deploymentManifest = json.dump(deploymentManifest, f, indent=4, sort_keys=True)

In [ ]:
# We have a deployment template file which needs to be updated with AI model's container registry address and credentials
file = open(deploymentManifestFileName)
deploymentTemplate = file.read()

if useACRFlag:
    deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_NAME', containerRegServiceName)
    deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_FULL_NAME', containerRegServiceName+".azurecr.io")
else:
    deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_NAME', localContainerRegServiceName)
    deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_FULL_NAME', localContainerRegServiceName)

deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_USER_NAME', acrUserName)
deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_PASSWORD', acrPassword)

# Save the resulting deployment file on disk, so we can pass it to deployment API (currently not supporting direct Json string input)
with open(deploymentManifestFileName, 'wt', encoding='utf-8') as outputFile:
    outputFile.write(deploymentTemplate)
    
print(deploymentTemplate)